In [1]:
import os
import pandas as pd
import openai
import numpy as np
import tensorflow as tf
import regex as re
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
import json

pd.set_option('display.max_colwidth', None)



In [2]:
basepath = '../../datasets/preprocessed_datasets/retailrocket/'
product_data = pd.read_pickle(basepath +'item_data_extracted.pkl')
product_data = product_data.reset_index(drop=True)
# product_data = product_data.sample(n=10, random_state=2).reset_index(drop=True)
display(product_data)

,itemid,categoryid,available,properties,property_values
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189, 202, 225, 227, 283, 364, 42, 561, 6, 678, 698, 764, 776, 790, 839, 869, 888, 917]","[1276750, n3.168 1144008, n15360.000 628176 n12288.000, 679677, 1168476, n4800.000 270060 924073, 519769, n96.000 1206660, 708480, 478989, 1301326, 1152934 1238769, 66094 372274 478989, 20716, n204.000, 1294803 101489, [66094, 66094, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769], 372274, 1152934 1238769, 1285872, 318611, n91200.000, 372274, 769062, 478989, 478989]"
1,1,1114,1,"[0, 1036, 112, 159, 185, 202, 227, 238, 280, 283, 284, 296, 33, 364, 438, 59, 6, 606, 664, 678, 689, 698, 730, 764, 768, 776, 790, 813, 839, 888, 917, 93, 981]","[769062, 1154859, 679677, 519769, 769062, 172646, 1020281 150169 1263524, 769062, 769062, 513325 1020281 1204938 172646 72261 30603 898004 140719 302848 1204938 190728 532620 572715 1128577 162103 1097148 288731 418473 150169 855390 836649 113294 435459 1103229 1019257 1272915 707081 13541 1161933 610340 804917, 769062, 866110, 1128577 1000087 421694, 1299107, 807503 1210748, 769062, 1020281, 769062, 992329, 1204938, 150169 1263524, 1020281, 769062, 1285872, n36.000, 379040, [n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6000.000, n6000.000, n6240.000, n5760.000, n5760.000, n5760.000, n5760.000, n5760.000], 814966, 1204938, [172646 1154859, 172646 1154859, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646], n372361069212.000, 769062, 769062]"
2,2,1305,0,"[1063, 112, 159, 202, 282, 283, 318, 332, 364, 443, 641, 678, 698, 699, 764, 776, 778, 790, 839, 877, 888, 917]","[n396.000 145688, 679677, 519769, 648485 n6000.000 n72.000, n192.000 145688, 822092 325894 504272 147366 343631 648485 n6000.000 n72.000 504272 147366 343631 648485 n6000.000 639502 n72.000 936990 949802 1113810 504272 653611 325894 1262898 796845 161652 150169 353870 1309799 1030323 608362 680618 1128577 636684 1081984 703408, 822092 325894, n72.000, 1260440, n6000.000 1144008, n288.000 145688, 147366 343631, 822092 325894 504272, n14616.000 10317, 1285872, 1005623, n840.000 1148727 237874, n41040.000, 147366 343631, n32.400 396934, 648485 n6000.000 n72.000 n72.000 468140, 648485 n6000.000 639502 n72.000]"
3,3,1171,0,"[1025, 1080, 112, 159, 202, 227, 250, 283, 30, 326, 33, 364, 453, 459, 562, 6, 604, 658, 678, 689, 698, 764, 776, 790, 839, 888, 917]","[769062, 769062, 679677, 519769, 261419, 1305767, 300154, [1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 1305767 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419, 138228 150169 1182824 327918 261419], 769062, 769062, 279887 421694, 354388, 769062, 769062, 769062, 1305767, 769062, 769062, 327918, 150169 1182824, 1305767, 1285872, 284910, n1560.000, 327918, 261419, 261419 829653]"
4,4,1038,0,"[112, 115, 159, 202, 227, 28, 283, 33, 364, 591, 6, 678, 689, 698, 764, 776, 790, 839, 841, 888, 897, 917, 981]","[679677, n24.000, 519769, 371058 71429, 588652 1091491, 150169 176547, 588652 768453 508476 371058 71429, 1128577 1000087 421694, 1142657, 1116693, 588652 1091491, 508476, 371058 150169 176547, 588652 10914

In [3]:
product_data['merged_properties'] = [
    {key: value for key, value in zip(row['properties'], row['property_values'])}
    for _, row in product_data.iterrows()
]

# Output the resulting dictionary
display(product_data)


,itemid,categoryid,available,properties,property_values,merged_properties
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189, 202, 225, 227, 283, 364, 42, 561, 6, 678, 698, 764, 776, 790, 839, 869, 888, 917]","[1276750, n3.168 1144008, n15360.000 628176 n12288.000, 679677, 1168476, n4800.000 270060 924073, 519769, n96.000 1206660, 708480, 478989, 1301326, 1152934 1238769, 66094 372274 478989, 20716, n204.000, 1294803 101489, [66094, 66094, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769, 1152934 1238769], 372274, 1152934 1238769, 1285872, 318611, n91200.000, 372274, 769062, 478989, 478989]","{'1036': '1276750', '1056': 'n3.168 1144008', '11': 'n15360.000 628176 n12288.000', '112': '679677', '127': '1168476', '139': 'n4800.000 270060 924073', '159': '519769', '177': 'n96.000 1206660', '189': '708480', '202': '478989', '225': '1301326', '227': '1152934 1238769', '283': '66094 372274 478989', '364': '20716', '42': 'n204.000', '561': '1294803 101489', '6': ['66094', '66094', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769', '1152934 1238769'], '678': '372274', '698': '1152934 1238769', '764': '1285872', '776': '318611', '790': 'n91200.000', '839': '372274', '869': '769062', '888': '478989', '917': '478989'}"
1,1,1114,1,"[0, 1036, 112, 159, 185, 202, 227, 238, 280, 283, 284, 296, 33, 364, 438, 59, 6, 606, 664, 678, 689, 698, 730, 764, 768, 776, 790, 813, 839, 888, 917, 93, 981]","[769062, 1154859, 679677, 519769, 769062, 172646, 1020281 150169 1263524, 769062, 769062, 513325 1020281 1204938 172646 72261 30603 898004 140719 302848 1204938 190728 532620 572715 1128577 162103 1097148 288731 418473 150169 855390 836649 113294 435459 1103229 1019257 1272915 707081 13541 1161933 610340 804917, 769062, 866110, 1128577 1000087 421694, 1299107, 807503 1210748, 769062, 1020281, 769062, 992329, 1204938, 150169 1263524, 1020281, 769062, 1285872, n36.000, 379040, [n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6120.000, n6000.000, n6000.000, n6240.000, n5760.000, n5760.000, n5760.000, n5760.000, n5760.000], 814966, 1204938, [172646 1154859, 172646 1154859, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646, 172646], n372361069212.000, 769062, 769062]","{'0': '769062', '1036': '1154859', '112': '679677', '159': '519769', '185': '769062', '202': '172646', '227': '1020281 150169 1263524', '238': '769062', '280': '769062', '283': '513325 1020281 1204938 172646 72261 30603 898004 140719 302848 1204938 190728 532620 572715 1128577 162103 1097148 288731 418473 150169 855390 836649 113294 435459 1103229 1019257 1272915 707081 13541 1161933 610340 804917', '284': '769062', '296': '866110', '33': '1128577 1000087 421694', '364': '1299107', '438': '807503 1210748', '59': '769062', '6': '1020281', '606': '769062', '664': '992329', '678': '1204938', '689': '150169 1263524', '698': '1020281', '730': '769062', '764': '1285872', '768': 'n36.000', '776': '379040', '790': ['n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6120.000', 'n6000.000', 'n6000.000', 'n6240.000', 'n5760.000', 'n5760.000', 'n5760.000', 'n5760.000', 'n5760.000'], '813': '814966', '839': '1204938', '888': ['172646 1154859', '172646 1154859', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646', '172646'], '917': 'n372361069212.000', '93': '769062', '981': '769062'}"
2,2,1

In [4]:
def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def combine_columns(row):
    return preprocess_text (
        "ID: " + str(row.itemid) + "; " +
        "categoryid: " + str(row.categoryid) + "; " +
        "available: " + row.available + "; " +
        "properties: " + str(row.merged_properties) + "; " 
    )


product_data_embeddings = product_data[['itemid']].copy()
product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
display(product_data_embeddings)

,itemid,combined
0,0,id 0 categoryid 209 available 0 properties 1036 1276750 1056 n3168 1144008 11 n15360000 628176 n12288000 112 679677 127 1168476 139 n4800000 270060 924073 159 519769 177 n96000 1206660 189 708480 202 478989 225 1301326 227 1152934 1238769 283 66094 372274 478989 364 20716 42 n204000 561 1294803 101489 6 property\n6 66094\n6 66094\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\n6 1152934 1238769\nname value dtype object 678 372274 698 1152934 1238769 764 1285872 776 318611 790 n91200000 839 372274 869 769062 888 478989 917 478989
1,1,id 1 categoryid 1114 available 1 properties 0 769062 1036 1154859 112 679677 159 519769 185 769062 202 172646 227 1020281 150169 1263524 238 769062 280 769062 283 513325 1020281 1204938 172646 72261 30603 898004 140719 302848 1204938 190728 532620 572715 1128577 162103 1097148 288731 418473 150169 855390 836649 113294 435459 1103229 1019257 1272915 707081 13541 1161933 610340 804917 284 769062 296 866110 33 1128577 1000087 421694 364 1299107 438 807503 1210748 59 769062 6 1020281 606 769062 664 992329 678 1204938 689 150169 1263524 698 1020281 730 769062 764 1285872 768 n36000 776 379040 790 property\n790 n6120000\n790 n6120000\n790 n6120000\n790 n6120000\n790 n6120000\n790 n6120000\n790 n6120000\n790 n6120000\n790 n6120000\n790 n6120000\n790 n6000000\n790 n6000000\n790 n6240000\n790 n5760000\n790 n5760000\n790 n5760000\n790 n5760000\n790 n5760000\nname value dtype object 813 814966 839 1204938 888 property\n888 172646 1154859\n888 172646 1154859\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\n888 172646\nname value dtype object 917 n372361069212000 93 769062 981 769062
2,2,id 2 categoryid 1305 available 0 properties 1063 n396000 145688 112 679677 159 519769 202 648485 n6000000 n72000 282 n192000 145688 283 822092 325894 504272 147366 343631 648485 n6000000 n72000 504272 147366 343631 648485 n6000000 639502 n72000 936990 949802 1113810 504272 653611 325894 1262898 796845 161652 150169 353870 1309799 1030323 608362 680618 1128577 636684 1081984 703408 318 822092 325894 332 n72000 364 1260440 443 n6000000 1144008 641 n288000 145688 678 147366 343631 698 822092 325894 504272 699 n14616000 10317 764 1285872 776 1005623 778 n840000 1148727 237874 790 n41040000 839 147366 343631 877 n32400 396934 888 648485 n6000000 n72000 n72000 468140 917 648485 n6000000 639502 n72000
3,3,id 3 categoryid 1171 available 0 properties 1025 769062 1080 769062 112 679677 159 519769 202 261419 227 1305767 250 300154 283 property\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 1305767 150169 1182824 327918 261419\n283 138228 150169 1182824 327918 261419\n283 138228 150169 1182824 327918 261419\n283 138228 150169 1182824 327918 261419\n283 138228 150169 1182824 327918 261419\n283 138228 150169 1182824 327918 261419\n283 138228 150169 1182824 327918 261419\n283 138228 150169 1182824 327918 261419\nname value dtype object 30 769062 326 769062 33 279887 421694 364 354388 453 769062 459 769062 562 769062 6 1305767 604 769062 658 769062 678 327918 689 150169 1182824 698 1305767 764 1285872 776 284910 790 n1560000 839 327918 888 261419 917 261419 829653
4,4,id 4 categoryid 1038 available 0 properties 112 679677 115 n24000 159 519769 202 371058 71429 227 588652 1091491 28 150169 

In [5]:
vectorizer = TextVectorization(output_mode='int', output_sequence_length=10)

# Apply the custom preprocessing function to the 'comb
# Update the vocabulary based on the preprocessed text
vectorizer.adapt(product_data_embeddings['combined'].values)

2023-08-06 10:04:30.506578: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [6]:
import os
import requests
import zipfile
import numpy as np

def download_and_extract_glove(glove_url, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    glove_zip = os.path.join(save_dir, 'glove.6B.zip')
    glove_txt = os.path.join(save_dir, 'glove.6B.100d.txt')

    if not os.path.exists(glove_txt):
        print("Downloading GloVe embeddings...")
        response = requests.get(glove_url, stream=True)
        with open(glove_zip, 'wb') as f:
            f.write(response.content)

        print("Extracting GloVe embeddings...")
        with zipfile.ZipFile(glove_zip, 'r') as zip_ref:
            zip_ref.extractall(save_dir)

        os.remove(glove_zip)

# Replace 'save_directory' with the path to the directory where you want to save the embeddings
save_directory = '../gabor'
glove_url = 'https://nlp.stanford.edu/data/glove.6B.zip'
download_and_extract_glove(glove_url, save_directory)

In [7]:
glove_txt = os.path.join(save_directory, 'glove.6B.100d.txt')
embeddings_index = {}
with open(glove_txt, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Define the Embedding layer using the pre-trained GloVe embeddings
vocab_size = len(vectorizer.get_vocabulary())
embedding_dim = 100  # You can choose the desired embedding dimension
embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = {word: index for index, word in enumerate(vectorizer.get_vocabulary())}

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False
)



In [8]:
# Convert the preprocessed text to numeric vectors
numeric_vectors = vectorizer(product_data_embeddings['combined'].values)

# Get word embeddings
word_embeddings = embedding_layer(numeric_vectors)
word_embeddings, word_embeddings.shape

(<tf.Tensor: shape=(417053, 10, 100), dtype=float32, numpy=
 array([[[-0.22458  , -0.58967  ,  0.43007  , ..., -0.63613  ,
           0.31284  , -0.82294  ],
         [-0.61491  ,  0.92734  ,  0.55826  , ..., -0.092153 ,
          -0.079833 , -1.2843   ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.56486  , -0.32277  , -0.1575   , ...,  0.0055935,
          -0.058519 , -0.89796  ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.48979  , -0.25559  ,  0.016606 , ...,  0.19874  ,
          -0.14428  , -0.8077   ]],
 
        [[-0.22458  , -0.58967  ,  0.43007  , ..., -0.63613  ,
           0.31284  , -0.82294  ],
         [-0.32219  ,  0.64632  ,  0.027312 , ..., -0.18124  ,
           0.50172  , -0.74622  ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [-0.61491  ,  0.92

In [9]:
# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
from sklearn.metrics.pairwise import cosine_similarity

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    input_embedding_flat = query_embedding.numpy().reshape(query_embedding.shape[0], -1)
    word_embeddings_flat = neighbors_embedding.numpy().reshape(neighbors_embedding.shape[0], -1)
    display(input_embedding_flat.shape, word_embeddings_flat.shape)
    # Calculate the cosine similarity between the input embedding and all other word embeddings
    similarities = cosine_similarity(input_embedding_flat, word_embeddings_flat)
    # Get the indices of the most similar neighbors
    neighbor_indices = np.argsort(similarities[0])[::-1][:n]

    scores = [similarities[0][i] for i in neighbor_indices]

    return neighbor_indices, scores


indices = get_n_nearest_neighbors(tf.gather(word_embeddings, [0]), word_embeddings)
display(indices)

(1, 1000)

(417053, 1000)

(array([     0,     85,    164,    227, 364634]),
 [1.0000001, 0.91049, 0.8963722, 0.88839453, 0.8857634])

In [10]:
product_data_embeddings.iloc[indices].combined

0                id 0 categoryid 209 available 0 properties 1036 1276750 1056 n3168 1144008 11 n15360000 628176 n12288000 112 679677 127 1168476 139 n4800000 270060 924073 159 519769 177 n96000 1206660 189 708480 202 478989 225 1301326 227 1152934 1238769 283 66094 372274 478989 364 20716 42 n204000 561 1294803 101489 6 property\n6              66094\n6              66094\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\n6    1152934 1238769\nname value dtype object 678 372274 698 1152934 1238769 764 1285872 776 318611 790 n91200000 839 372274 869 769062 888 478989 917 478989 
85                id 91 categoryid 209 available 0 properties 1036 726612 1056 769062 11 n30720000 628176 n17280000 112 679677 127 1983

In [ ]:
product_data.iloc[indices]

In [ ]:
product_data